## Web content search/query through LLM
#### Retrieval augmented generation, or RAG
- Load web content using `WebBaseLoader` and `bs4`
- Create docs and divide into chunks using `RecursiveCharacterTextSplitter`
- Initialize embedding `OpenAIEmbeddings` to create vector embeddings
- Store to document chunks as vector store db (`Chroma`, `FAISS`) 
- Import and intialize LLM - `ChatOpenAI` with `gpt-4o` model
- Create prompt template
- Create chain to

**PromptTemplate** is used to dynamically generate LLM prompts by embedding variables into a predefined structure
- ChatPromptTemplate.`from_template(prompt)`
    - Creates a chat template consisting of a single message assumed to be from the human.
    - `prompt = """Suggest a restaurant name in {country} that serves {cuisine} food."""`
- ChatPromptTemplate.`from_messages(prompt)`
    - Create a chat prompt template from a variety of message formats.
    - `prompt= [("system", "Act like a doctor and answer my questions"),("user", "{input}")`

**Retriever** is an interface for fetching or retrieving relevant data from documents(chunks) or vector store

**Chain** refers to a sequence of steps or components that are linked together to perform a more complex task using a language model.
- Example - `prompt-template | llm | output-parser`


#### Load -> Chunk -> Embed -> Store

In [40]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from IPython.display import Markdown

loader = WebBaseLoader("https://www.sobha.com/blog/bangalore-real-estate-market-trends/")
documents = loader.load()

sp = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_chunks = sp.split_documents(documents)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(doc_chunks, embeddings)

Markdown(db.similarity_search('Why is Bangalore rent so costly?')[0].page_content)


7. Why is real estate booming in Bangalore? 

Real estate is booming in Bangalore due to demand driven by the city's expanding population and relatively lower expense of living. According to Bangalore real estate trends 2024, many people are looking for opulent and easily accessible dwelling options. 





8. Will flat prices go up in Bangalore 

Yes, flat prices could go up in Bangalore, as the city has witnessed the most increase in house costs, at 7%, while Mumbai saw a 6% increase. In contrast, prices in Chennai and Hyderabad increased by 5% each in the first quarter of 2023. 





9. Why is house rent so expensive in Bangalore? 

House rent is so expensive in Bangalore due to employees’ return to work-from-office, creating a spike in demand for housing. Another reason is higher commute times that have forced professionals to pick residences close to their workplaces, typically concentrated in particular locations, increasing the demand for apartments.

In [41]:
Markdown(db.similarity_search('who is the foreign minister of Bangladesh?')[0].page_content)

Comprehensive Urban Plan
The Revised Master Plan 2041 aims to transform Bangalore into a ‘compact city’ by promoting development around public transport systems. This plan includes significant upgrades to metro and road infrastructure, ensuring enhanced support for the city’s growth.
These infrastructure projects will drive the demand for housing, with estimates suggesting that more than 100,000 new residential units will be required by 2025. Improved connectivity and reduced travel time will make the city an even more attractive destination for businesses and residents alike.

#### Chat Prompt template

In [42]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the given context:
    <context>
    {context}
    </context>
    """
)

#### Document Chain
- *Processes a series of documents or text inputs to produce an output in stages*
- *Useful for summarization, translation, or other step-by-step transformations.*
- Stages - `Prompt Template -> LLM -> Output Parser`

In [43]:
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI(model="gpt-4o")
doc_chain = create_stuff_documents_chain(llm, prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the following question based on the given context:\n    <context>\n    {context}\n    </context>\n    '))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025030A13730>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025030A35720>, root_client=<openai.OpenAI object at 0x0000025030A36C80>, root_async_client=<openai.AsyncOpenAI object at 0x0000025030A137C0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

#### Vector Store Retriver and Retrival Chain
- *To retrieve the context information through a Retriver (here it's from vector store db)*

In [44]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
# Used to retrieve from vector store
chain = create_retrieval_chain(retriever, doc_chain)
chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025030A126E0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the following question based on the given context:\n    <context>\n    {context}\n    </context>\n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025030A13730>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0

#### Query

In [46]:
response = chain.invoke({"input": "Is Bangalore rent going increase?"})
Markdown(response['answer'])

Real estate is booming in Bangalore due to the city's expanding population and relatively lower expense of living. Many people are also seeking opulent and easily accessible dwelling options, which further drives the demand.

In [48]:
response = chain.invoke({"input": "Which areas are good for residential property investments?"})
Markdown(response['answer'])

Based on the given context, answer the following question:

**Which areas in Bangalore are emerging as top investment zones due to their robust infrastructure, commercial vitality, and vibrant communities?**

**Answer:**
Areas such as Sarjapur Road, Bellandur, and Yelahanka are emerging as top investment zones in Bangalore due to their robust infrastructure, commercial vitality, and vibrant communities.

In [55]:
response = chain.invoke({"input": "Which are major infra developments coming up?"})
Markdown(response['answer'])

Which infrastructure projects in Bangalore are expected to enhance connectivity and support real estate development?

The infrastructure projects in Bangalore expected to enhance connectivity and support real estate development include:

1. **Bangalore Satellite Town Ring Road (STRR)**
2. **Bangalore-Mysore Expressway**
3. **Peripheral Ring Road**
4. **NICE Road**
5. **Bangalore-Chennai Expressway**
6. **New metro lines connecting central business districts to Kempegowda International Airport**
7. **Blue Line metro connecting Silk Board to Kempegowda International Airport (expected by mid-2026)**

These projects aim to reduce travel time and congestion, making office locations more accessible and attractive, thus boosting the commercial real estate market.